Title: Interpolation Methods with ArcPy: Interpolating Elevation

Course: GIS 5572: ArcGIS II

Author(s): Mattie Gisselbeck

Date: 3-25-2023

Abstract

Previously, we built a pipeline that (1) extracts, transforms, and loads data; (2) performs QAQC operations on the imported data; (3) saves the data to a local geodatabase; and (4) then saves it to a PostgresSQL database hosted on Google Cloud. The objective of this lab was to create interpolated temperature maps for the state of Minnesota and evaluate their accuracy using the ETL and QAQC pipeline. The resulting maps and accuracy assessment will be stored in a local geodatabase and PostgresSQL database. The interpolated maps will be viewable on ArcGIS Online MapViewer via GeoJSON from a Flask API endpoint.

https://test11-pmz7lxrsca-uc.a.run.app

In [7]:
import arcpy
import requests
import os
import psycopg2
import random
from pathlib import Path

In [ ]:
os.chdir(r"\\Mac\Home\Documents\git")
wksp = os.getcwd()

1. Querying Elevation Data from PostgresSQL Database

In [ ]:
# Execute a Query in PostgresSQl Database and Create a Layer
arcpy.management.MakeQueryLayer(
    input_database=os.path.join(wksp, ""),
    out_layer_name="elevation",
    query="SELECT * FROM elevation;",
    oid_fields="pointid",
    shape_type="POINT",
    srid="4326",
    spatial_reference='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision',
    spatial_properties="DEFINE_SPATIAL_PROPERTIES",
    m_values="DO_NOT_INCLUDE_M_VALUES",
    z_values="DO_NOT_INCLUDE_Z_VALUES",
    extent="0 0 0 0"
)

# Copy Features from Query Layer (Converts Layer to .shp)
arcpy.management.CopyFeatures(
    in_features="elevation",
    out_feature_class=os.path.join(wksp, "DEM.shp"),
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

2. Sampling Elevation Data

In [ ]:
# Define Input .shp Path
dem = "DEM.shp"

# Define Output .shp for DEM and DEM Reference Samples
dem_sample = "DEM_Sample.shp"
dem_reference_sample = "DEM_ReferenceSample.shp"

In [ ]:
# Set Sample Percentage
sample_percentage = 1

# Count Number of Features in DEM.shp
feature_count = int(arcpy.GetCount_management(dem).getOutput(0))
feature_count_reference = int(arcpy.GetCount_management(dem).getOutput(0))

# Calculate Number of Features to Sample
sample_count = int((sample_percentage / 100) * feature_count)
sample_count_reference = int((sample_percentage / 100) * feature_count_reference)

In [ ]:
# Generate a List of Random Feature ID(s) for Sample
id_list = random.sample(range(1, feature_count+1), sample_count)

# Generate a List of Random Feature id(s) for Reference Sample
id_reference_list = random.sample(range(1, feature_count_reference+1), sample_count_reference)

# Convert the Lists to Sets and Find Common Points
common_elements = set(id_list) & set(id_reference_list)

# Remove common points from reference sample list
id_reference_list = [item for item in id_reference_list if item not in common_elements]

In [ ]:
# Convert the Lists to Comma-Separated Strings
id_str = ",".join(str(x) for x in id_list)
id_reference_str = ",".join(str(x) for x in id_reference_list)

# Create a SQL Query to Select Randomly Sampled Features for DEM Sample
dem_sample_query = '"FID" IN ({})'.format(id_str)

# Create a SQL Query to Select Randomly Sampled Features for DEM Reference Sample
dem_reference_query = '"FID" IN ({})'.format(id_reference_str)

# Use Select_analysis Tool to Select Features and Create New .shp Files
arcpy.Select_analysis(dem, dem_sample, dem_sample_query)
arcpy.Select_analysis(dem, dem_reference_sample, dem_reference_query)

3. Interpolating Elevation using ArcPy

* **IDW()** uses the measured values surrounding the prediction location to predict a value for any unsampled location, based on the assumption that things that are close to one another are more alike than those that are farther apart.
* **GlobalPolynomialInterpolation()** fits a smooth surface that is defined by a mathematical function (a polynomial) to the input sample points.
* **EmpiricalBayesianKriging()** is an interpolation method that accounts for the error in estimating the underlying semi-variogram through repeated simulations.

In [6]:
arcpy.ddd.Idw(
    in_point_features=elev_sample,
    z_field="grid_code",
    out_raster=os.path.join(wksp, "IDW_DEM.tif"),
    cell_size=0.1,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)

arcpy.ddd.Kriging(
    in_point_features=elev_sample,
    z_field="grid_code",
    out_surface_raster=os.path.join(wksp, "Kriging_DEM.tif"),
    semiVariogram_props="Spherical 0.021245 # # #",
    cell_size=0.1,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

arcpy.ga.GlobalPolynomialInterpolation(
    in_features=elev_sample,
    z_field="grid_code",
    out_ga_layer=None,
    out_raster=os.path.join(wksp, "GPI_DEM.tif"),
    cell_size=0.1,
    power=1,
    weight_field=None
)

<Result 'DEM_IDW_GeostatisticalLayer'>

4. Creating an Accuracy Assessment

In [ ]:
def create_accuracy_assessment (raster, validation_data):

     # Define Output Path and Name of Ground Truth vs. Classified .shp
    output_acc = Path(raster).stem + '_PointAccuracy' + '.shp'
    acc_table = os.path.join(wksp, output_acc)

    # Define Output Path and Name, Saves RMSE for Each Interpolation
    output_stat = Path(raster).stem + '_Statistics.dbf'
    stat_table = os.path.join(wksp, output_stat)

    # Extract Predicted Values and Save to Validation Data
    arcpy.sa.ExtractValuesToPoints(
        in_point_features=validation_data,
        in_raster=raster,
        out_point_features=acc_table,
        interpolate_values="NONE",
        add_attributes="ALL"
    )
    # Rename Default Fields
    arcpy.management.CalculateField(
        in_table=acc_table,
        field="GrndTruth",
        expression="!min_tmpf!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    arcpy.management.CalculateField(
        in_table=acc_table,
        field="Classified",
        expression="!RASTERVALU!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    arcpy.management.DeleteField(
        in_table=acc_table,
        drop_field="min_tmpf;RASTERVALU",
        method="DELETE_FIELDS"
    )

    # Calculate Squared Error
    arcpy.management.CalculateField(
        in_table=acc_table,
        field="Sq_error",
        expression="math.pow(!GrndTruth! - !Classified!, 2)",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

    # Create Statistics Table and Calculate Squared Error Sum
    arcpy.analysis.Statistics(
        in_table=acc_table,
        out_table=stat_table,
        statistics_fields="Sq_error SUM",
        case_field=None,
        concatenation_separator=""
    )
    arcpy.management.CalculateField(
        in_table=stat_table,
        field="RMSE",
        expression="math.sqrt(!SUM_Sq_err! / !FREQUENCY!)",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

5. Exploratory Interpolation

In [8]:
# Run Exploratory Interpolation
arcpy.ga.ExploratoryInterpolation(
    in_features = "DEM_RastertoPoint",
    value_field = "pointid",
    out_cv_table = r"\\Mac\Home\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\DEM_ExploratoryInterpolation",
    out_geostat_layer = "DEM_GeostatisticalLayerHighestRank",
    interp_methods = "EBK;IDW;GPI",
    comparison_method = "SINGLE",
    criterion = "ACCURACY",
    criteria_hierarchy = "ACCURACY PERCENT #",
    weighted_criteria = "ACCURACY 1",
    exclusion_criteria = None
)

# Export Table as .dbf
arcpy.conversion.ExportTable(
    in_table = "DEM_ExploratoryInterpolation",
    out_table = r"\\Mac\Home\Desktop\DEM_ExpolatoryInterpolation.dbf",
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
    field_mapping = 'DESCR "Model Description" true true true 255 Text 0 0,First,#,DEM_ExploratoryInterpolation,DESCR,0,255;RANK "Rank" true true true 4 Long 0 0,First,#,DEM_ExploratoryInterpolation,RANK,-1,-1;INCLUDED "Included" true true true 255 Text 0 0,First,#,DEM_ExploratoryInterpolation,INCLUDED,0,255;RMSE "Root Mean Square Error" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,RMSE,-1,-1;ME "Mean Error" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,ME,-1,-1;ME_STD "Mean Standardized Error" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,ME_STD,-1,-1;RMSE_STD "Root Mean Square Standardized Error" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,RMSE_STD,-1,-1;ASE "Average Standard Error" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,ASE,-1,-1;MAX_ERROR "Maximum Absolute Error" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,MAX_ERROR,-1,-1;PERC_ERROR "Percent Error Reduction" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,PERC_ERROR,-1,-1;CRPS "Average CRPS" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,CRPS,-1,-1;PERC_90 "Inside 90 Percent Interval" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,PERC_90,-1,-1;PERC_95 "Inside 95 Percent Interval" true true true 8 Double 0 0,First,#,DEM_ExploratoryInterpolation,PERC_95,-1,-1',
    sort_field = None
)

# Read .dbf File
expolatory_interpolation = r"/Users/mattiegisselbeck/Desktop/DEM_ExpolatoryInterpolation.dbf"
df = gpd.read_file(expolatory_interpolation)
df

,DESCR,RANK,INCLUDED,RMSE,ME,ME_STD,RMSE_STD,ASE,MAX_ERROR,PERC_ERROR,CRPS,PERC_90,PERC_95,geometry
0,Empirical Bayesian Kriging - Advanced,1,Yes,0.224318,-0.014816,-0.046929,0.421152,0.390328,10.419940,99.997268,0.087951,99.838284,99.933204,None
1,Empirical Bayesian Kriging - Default,2,Yes,1.268946,-0.904460,-0.106932,0.144849,8.430596,18.220115,99.984547,1.976801,99.992969,99.996484,None
2,Inverse Distance Weighted - Optimized,3,Yes,9.899689,-0.040151,0.000000,0.000000,0.000000,124.500000,99.879441,0.000000,0.000000,0.000000,None
3,Inverse Distance Weighted - Default,4,Yes,23.355296,-0.074910,0.000000,0.000000,0.000000,244.125428,99.715578,0.000000,0.000000,0.000000,None
4,Global Polynomial Interpolation – Third order,5,Yes,390.482435,-0.026003,0.000000,0.000000,0.000000,3988.855961,95.244693,0.000000,0.000000,0.000000,None
5,Global Polynomial Interpolation – Second order,6,Yes,420.537546,-0.009746,0.000000,0.000000,0.000000,3408.102384,94.878681,0.000000,0.000000,0.000000,None


In [ ]:
# Create Lists with File Names With and Without .tif Extension
interpolations = ['DEM_IDW.tif', 'DEM_EBK.tif', 'DEM_GPI.tif']
interpolators  = ['DEM_IDW', 'DEM_EBK', 'DEM_GPI']

# Create Accuracy Assessments for Each Interpolation
for i in range(len(interpolations)):
    accuracy_assessment(interpolations[i], dem)

# Merge Accuracy Assessments
arcpy.management.Merge(
    inputs="AAT_IDW_DEM;AAT_Kriging_DEM;AAT_GPI_DEM_stat",
    output="DEM_AccuracyAssessmentTable.dbf",
    field_mappings='Interpolat "Interpolat" true true false 255 Text 0 0,First,#;FREQUENCY "FREQUENCY" true true false 10 Long 0 10,First,#,Acc_IDW_stat,FREQUENCY,-1,-1,Acc_Kriging_stat,FREQUENCY,-1,-1,Acc_GPI_stat,FREQUENCY,-1,-1;SUM_Sq_err "SUM_Sq_err" true true false 19 Double 0 0,First,#,Acc_IDW_stat,SUM_Sq_err,-1,-1,Acc_Kriging_stat,SUM_Sq_err,-1,-1,Acc_GPI_stat,SUM_Sq_err,-1,-1;RMSE "RMSE" true true false 13 Float 0 0,First,#,Acc_IDW_stat,RMSE,-1,-1,Acc_Kriging_stat,RMSE,-1,-1,Acc_GPI_stat,RMSE,-1,-1',
    add_source="NO_SOURCE_INFO"
)

# Add Name of Each Interpolator to Merged Table
with arcpy.da.UpdateCursor("DEM_AccuracyAssessmentTable.dbf", ['Interpolat']) as cursor:
    for i, row in enumerate(cursor):
        if i < len(interpolators):
            row[0] = interpolators[i]
        else:
            break
        cursor.updateRow(row)

# Delete Cursor to Release Locks on Data
del cursor

In [ ]:
# Find Interpolator with the Lowest Root Mean Square Error (RMSE)
methods = {}
fields = ["pointid", "RMSE"]
with arcpy.da.SearchCursor('DEM_AccuracyAssessmentTable.dbf', fields) as cursor:
    for row in cursor:
        methods[row[0]] = row[1]

best_interpolator = min(methods, key=methods.get)

In [ ]:
# Clip to Minnesota State Boundary
output_clip = os.path.join(wksp, best_interpolator + '_Minnesota.tif')
out_raster = arcpy.sa.ExtractByMask(
    in_raster=best_interpolator+'.tif',
    in_mask_data="Minnesota_StateBoundary.shp",
    extraction_area="INSIDE",
    analysis_extent='-97.239102895829 43.499445217943 -89.6516983029999 49.0583312990001 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
)
out_raster.save(output_clip)

# Convert Raster to Point Shapefile
output_point_shp = os.path.join(wksp, best_interpolator + '.shp')
arcpy.conversion.RasterToPoint(
    in_raster=output_clip,
    out_point_features=output_point_shp,
    raster_field="Value"
)

6. Saving Layer(s) and Table(s) to PostgresSQL Database Using psycopg2

In [ ]:
# Establish Connection to PostgreSQL Database
connection = psycopg2.connect(
    host = '34.133.121.12',
    port = '5432',
    database = 'lab3',
    user = 'postgres',
    password = 'student',
)

6.1. Interpolation: DEM

In [ ]:
# Define Variables
point_table = best_interpolator.lower()
fields = ["pointid", "grid_code", "Shape@WKT"]

# Create Table with Best Interpolator
cursor = connection.cursor()
cursor.execute(f"DROP TABLE IF EXISTS {point_table}")
cursor.execute(f"""
    CREATE TABLE {point_table} (
        pointid INT,
        grid_code DOUBLE PRECISION)
""")

cursor.execute(f"""
    SELECT AddGeometryColumn('{point_table}', 'geom', 4326, 'POINT', 2)
""")

# Populate Table
with arcpy.da.SearchCursor(output_point_shp, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute(f"INSERT INTO {point_table} (pointid, grid_code, geom) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))

connection.commit()

6.2. Point Accuracy Assessment Table: DEM

In [ ]:
# Define Path of .dbf and Fields
accuracy_assessment = 'DEM_AccuracyAssessmentTable.dbf'
fields = ["OID", "Interpolat", "FREQUENCY", "SUM_Sq_err", "RMSE"]

# Create Table
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS dem_accuracyassessment")
cursor.execute("""
    CREATE TABLE dem_accuracyassessment (
        OID INT,
        Interpolat VARCHAR,
        FREQUENCY INT,
        SUM_Sq_err DOUBLE PRECISION,
        RMSE DOUBLE PRECISION)
""")

# Populate Table
with arcpy.da.SearchCursor(accuracy_assessment, fields) as da_cursor:
    for row in da_cursor:
        cursor.execute("INSERT INTO dem_accuracyassessment (OID, Interpolat, FREQUENCY, SUM_Sq_err, RMSE) VALUES (%s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4]))

connection.commit()

6.3. Point Accuracy Assessment Layer: DEM

In [ ]:
# Define Path of .shp
data = os.path.join(wksp, 'Acc_' + best_interpolator + '.shp')

# Create Table Name
table_name = best_interpolator.lower() + '_error_estimation'

fields = ["GrndTruth", "Classified", "Sq_error", "Shape@WKT"]

# Create Ttable
cursor = connection.cursor()
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
cursor.execute(f"""
    CREATE TABLE {table_name} (
        GrndTruth DOUBLE PRECISION,
        Classified DOUBLE PRECISION,
        Sq_error DOUBLE PRECISION)
""")

cursor.execute(f"""
    SELECT AddGeometryColumn('{table_name}', 'geom', 4326, 'POINT', 2)
""")

# Populate Table
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[3]
        cursor.execute(f"INSERT INTO {table_name} (GrndTruth, Classified, Sq_error, geom) VALUES (%s, %s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], wkt))

connection.commit()
connection.close()